# k-Means.

El algoritmo k-means es un método de clustering o agrupamiento de datos que se utiliza para clasificar conjuntos de datos no etiquetados en grupos o clústeres basados en la similitud de sus características.

Se le considera de aprendizaje no supervisado porque no necesita etiquetas para funcionar.

En Scikit-learn, la implementación de k-means se encuentra en la clase <code><b>KMeans</b></code>. 

In [ ]:
from sklearn.cluster import KMeans

Esta clase ofrece varias opciones de configuración, como la cantidad de clústeres que se desea encontrar, la inicialización de los centroides y el número máximo de iteraciones. Pero antes de verlas, te voy a enseñar su uso básico con el dataset de Iris (recuerda que son tres tipos de flores):

In [ ]:
from sklearn.datasets import load_iris

iris = load_iris()
X = iris.data

Para inicializar <code>KMeans</code> es necesario especificar el número de clusteres que queremos obtener, y esta es tal vez una de las debilidades del algoritmo: tienes que especificarle de antemano cuántos clústers necesitas – ya sabes que scikit-learn tiene valores por default para sus argumentos, el valor por default para este argumento es 8, pero nosotros vamos a dejarlo en 3:

In [ ]:
kmeans = KMeans(n_clusters=3)
kmeans.fit(X)

 > 📢 Además, recuerda que para que k-Means funcione correctamente, los datos deben estar en escalas similares, lo que quiere decir que debes tratar de escalar tus datos antes de introducirlos al modelo.

## Atributos

Una vez ya entrenado podemos encontrar los clústeres de los datos utilizando el atributo <code>labels_</code>:

In [ ]:
kmeans.labels_

También podemos acceder a los centroides que calculo, recuerda que hay tantos centroides como número de clústeres:

In [ ]:
kmeans.cluster_centers_

En este caso, tenemos 3 centroides de 4 dimensiones cada uno porque nuestros datos de entrada eran 4-dimensionales.

Pero es mejor visualizado en una gráfica (esta gráfica solamente utiliza un par de las características del dataset):

In [ ]:
from utils import view_centroids_iris

view_centroids_iris(kmeans, X)

Otro de los atributos es la inercia. La inercia mide la dispersión interna de los clusters, es decir qué tan lejos están los puntos del centroide más cercano. En general, el objetivo de k-Means es minimizar este valor. Una vez ya entrenado podemos acceder a esta información a través del atributo:

In [ ]:
kmeans.inertia_

## Argumentos de <code>kmeans</code>

El algoritmo KMeans tiene varios argumentos importantes que se pueden ajustar para obtener los resultados deseados. A continuación, te presento los argumentos más importantes:

 - <code><b>n_clusters</b></code>: Especifica el número de clústeres que se desean en la solución. Este es el parámetro más importante y debe ser ajustado cuidadosamente.

 - <code><b>init</b></code>: Especifica el método de inicialización de los centroides de los clústeres. Las opciones son "k-means++", "random" y un arreglo personalizado, "k-means++" es el método predeterminado y se recomienda para la mayoría de los casos.

 - <code><b>n_init</b></code>: Especifica el número de veces que el algoritmo se ejecutará con diferentes inicializaciones de centroides. La solución final será la mejor de todas las ejecuciones. El valor predeterminado es 10, pero se puede aumentar si se quiere encontrar una solución más precisa.

 - <code><b>max_iter</b></code>: Especifica el número máximo de iteraciones permitidas antes de que el algoritmo se detenga, incluso si no ha convergido. El valor predeterminado es 300.

 - <code><b>tol</b></code>: Especifica la tolerancia para la convergencia. Si la distancia entre el centroide y su centroide anterior es menor que <code><b>tol</b></code>, se considera que el algoritmo ha convergido. El valor predeterminado es <code>1e-4</code>.

## Jugando con los argumentos

In [ ]:
from utils import plot_centroids

import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs

X, y = make_blobs(n_samples=300, centers=6, cluster_std=1, random_state=42)

plt.scatter(X[:,0],X[:,1], c=y)

### <code>n_clusters</code>

Tal vez los valores más importantes para tunear sean la cantidad de clusters:

In [ ]:
# Variando n_clusters
n_clusters_list = [2, 3, 4, 5, 6, 7]

trained_kmeans = []
titles = []
for n_clusters in n_clusters_list:
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init='auto')
    kmeans.fit(X)
    trained_kmeans.append(kmeans)
    titles.append(f"n_clusters = {n_clusters}")
plot_centroids(input_features=X, trained_kmeans=trained_kmeans, titles=titles)

### Elbow method – la regla del codo

La gran mayoría de las veces es imposible visualizar los centroides de nuestros datos (por aquello de la gran dimensionalidad). Pero puedes hacer uso de “la regla del codo”. La del codo es una heurística usada para determinar el número óptimo de clusteres. Consiste en buscar un punto de inflexión en el que la inercia deja de cambiar drásticamente.

In [ ]:
inertias = [kmeans.inertia_ for kmeans in trained_kmeans]

fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(n_clusters_list, inertias , marker='o')
ax.set_xlabel('Número de clusters')
ax.set_ylabel('Inercia')
ax.set_title('Regla del codo')

Adicionalmente, recuerda que hay otras métricas que ya vimos previamente en el módulo de métricas de agrupamiento.

### <code>init</code>

La forma de inicializar los clusters

In [ ]:
init_methods = ['k-means++', 'random', np.array([[-10, -10] for _ in range(6)])]
init_titles = ['k-means++', 'random', 'custom']

trained_kmeans = []
titles = []
for title, init in zip(init_titles, init_methods):
    kmeans = KMeans(n_clusters=6, init=init, random_state=42, n_init=1)
    kmeans.fit(X)
    trained_kmeans.append(kmeans)
    titles.append(f"init = {title}")
plot_centroids(input_features=X, trained_kmeans=trained_kmeans, titles=titles)

## <code>max_iter</code>

El número máximo de iteraciones

In [ ]:
# Variando max_iter
max_iter_list = [1, 2, 3, 4, 5, 300]

initial_centroids = np.array([[0, 0] for _ in range(6)])

trained_kmeans = []
titles = []
for max_iter in max_iter_list:
    kmeans = KMeans(n_clusters=6, max_iter=max_iter, init=initial_centroids, n_init=1, random_state=42)
    kmeans.fit(X)
    trained_kmeans.append(kmeans)
    titles.append(f'max_iter: {max_iter}')
    
plot_centroids(input_features=X, trained_kmeans=trained_kmeans, titles=titles)

## Kmeans y grandes datasets

Kmeans es un algoritmo que funciona bien para datasets de tamaño moderado. Sin embargo se vuelve muy poco eficiente cuando se utiliza para datasets grandes, tanto en número de filas u observaciones, como en número de columnas o features.

Dentro de Scikit-Learn existe otro algoritmo llamado Mini-batch k-Means, que en lugar de operar sobre todo el dataset a la vez (como es el caso de kMeans) opera sobre un subconjunto de elementos a la vez.

 > 📚 De tarea, ¿por qué no lo usas y ves su comportamiento? lo puedes importar de <code>sklearn.cluster</code> como <code>MiniBatchKMeans</code>.

## En conclusión

KMeans es un algoritmo que puedes usar cuando:

 1. Necesitas agrupar datos no etiquetados en función de su similitud, ya que KMeans busca dividir el conjunto de datos en grupos (clusters) compactos y separados.

 1. Tienes un conjunto de datos de tamaño moderado y dimensionalidad no muy alta

 1. Deseas un algoritmo fácil de implementar y entender

Pero deberías tener cuidado de usarlo en:

 1. Datos con ruido, valores outliers o datos que se superponen entre diferentes grupos

 1. Datos de alta dimensionalidad, ya que KMeans puede verse afectado por la "maldición de la dimensionalidad"

 1. Conjuntos de datos extremadamente grandes, en cuyo caso podrías considerar el uso de Mini-Batch KMeans u otros algoritmos de clustering más escalables.

 1. Situaciones en las que no tienes una idea aproximada del número de clusters, ya que KMeans requiere que especifiques el número de clusters de antemano.
